![alt text](https://storage.googleapis.com/pr-newsroom-wp/1/2018/11/Spotify_Logo_CMYK_Green.png)

## Working with Spotify Data using SpotiPy
-Jeremy Seibert

Spotify is an amazing music streaming service that touches over 200 million customers. This is the notebook where I will show you how to work with the Spotify API using SpotiPy. This was the methodology that I used to create the datasets used in my spotify song prediction notebook. The SpotiPy package assists in the API connection to the Spotify backend's publicly available data. Once connected to the API, we can query an enormous amount of Spotify data related to their user, playlist, and song data. So lets jump in!

### 1. Import the Necessities

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd

### 2. Setup your Spotify App Credentials

As with most APIs, you will need to register with the service that you are wanting to connect too. This is exactly the case for Spotify. Spotify's Developers have made it easy for other developers to connect to the to their data. 

 1. You will first need to use your Spotify account to register for a developer account, or you can setup a free spotify account and then register for your developer account [Setup Spotify Dev Account here](https://developer.spotify.com/dashboard/). 

2. After registering setring up your developer account, you will need to Setup an App. 
    - Locate the Dashboard tab, and click it. 
    - Select "Create a Client ID"
    - Fill in the name of the App, in this case I just put "Notebook 1"
    - Fill in the App or Hardware Description, I put "Jupyter Notebook"
    - Select what you are building, I put "Website"
    - The question regarding a commercial integration simply means are yo trying to make money off of this? & the quick answer here is No (If you are a business, and are looking to integrate this to make money, check Yes)
    - Agree to the Terms of Service.
    
    
3. You should be redirected to your Dashboard, once there click the "Edit settings" button. 

4. In the Edit Settings popup, find the Redirect URIs blank and insert "http://google.com/" and MAKE SURE AT THE BOTTOM YOU CLICK SAVE. 

5. Go back to your dashboard an locate your Client ID and your Secret ID and fill them into the notebook. 

6. Your username is actually pretty easy to find. Go to your Spotify Application (The APP on you Computer, not the one we just created)

    - Step 1: Navigate to any playlist that you have created. Here I chose one of the playlists analyzed in the spotify prediction notebook. 
        
![alt text](./Documents/Step1.png)

  - Step 2: Click on the Three Dot buttion once on a playlist that you created. 
    
![alt text](./Documents/Step2.png)
    
   - Step 3: Paste the Copied URI into a Browser and there is your User ID and the Playlist ID.
![alt text](./Documents/Step3.png)
    
    

In [3]:
cid ='<Your App ID>'
secret = '<Your Secret App ID>'
username = "<Your User ID>"

### 3. Client Credential and Authorization Crediential Flow

The next couple steps cover how to implement the Authorization and Client Crediential Code Flow necessary to access Spotifys API. We use the Spotipy class SpotifyClientCredentials to authenticate our requests using the information gathered in the previous step. We then pass the infromation into an instance or the Spotipy class Spotify with the client_credentials_manager variable as an argument. 

The next step that we need to take is to define the Authorization Scope. Here we are using two different scopes, the "user-library-read" and the "playlist-read-private" scopes. The first allows users to access a user's "Your Music" library, and access their saved tracks and albums. The second, allows users	to access to a user's private playlists. Check out this to find more information about [Spotify's Authorization Scopes](https://developer.spotify.com/documentation/general/guides/scopes/)

Next, we setup the Authorization Credentials Flow, which is done by using the util.prompt_for_user_token method. We pass the username, scope, Client ID, Client Secret, and the redirect URI each as arguments into the prompt for user token method. The last step, is to verify that the token is not null, and run it.

In [ ]:
#SpotiPy Client Credentials Flow
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Sets the Authorization Scope
scope = 'user-library-read playlist-read-private'

# SpotiPy Authorization Credenitals Flow
token = util.prompt_for_user_token(username, scope,client_id=cid,client_secret=secret,redirect_uri='http://google.com/')

if token is not None:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=%3CYour+App+ID%3E&response_type=code&redirect_uri=http%3A%2F%2Fgoogle.com%2F&scope=playlist-read-private+user-library-read in your browser




### 4. Build a function to Get Track IDs 

This function will allow us to take a userID and playlistID and will return a list of the Spotify track IDs within the playlist. 

In [ ]:
def get_tracks(user,playlist):
    ''' This function takes a user key and playlistID and returns the trackIDs within that users playlist.'''
    uplaylist = sp.user_playlist(user,playlist)
    tracks = uplaylist["tracks"]
    songs = tracks["items"] 
    while tracks['next']:
        tracks = sp.next(tracks)
        for item in tracks["items"]:
            songs.append(item)
    ids = [songs[i]['track']['id'] for i in range(len(songs))]
    return ids

### 5. Build a function to Get Track Audio Features 

This function will allow us to take a trackIDs and will return the audio features for each track. The audio features include metrics such as loudness, dancability, among others. You can read more about them [here](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/).

In [1]:
def get_audio_features(ids):
    '''This function takes a track and gathers the track's Audio Features, and adds a new target feature column.'''
    features = []
    for i in range(0,len(ids),50):
        audio_features = sp.audio_features(ids[i:i+50])
        for track in audio_features:
            features.append(track)
    return features

Using each of these functions I pulled together the dataset that is used in the spotify predictions notebook. I only cover a small portion of the potential from the Spotify API, and the data that can be accessed. 

Here are some more great resources:

1. [Spotify Developer Site](https://developer.spotify.com/documentation/web-api/)

2. [SpotiPy Documents](https://spotipy.readthedocs.io/en/latest/)